## LLM RAG Evaluation with MLflow using llama2-as-judge Example Notebook

In this notebook, we will demonstrate how to evaluate various a RAG system with MLflow. We will use llama2-70b as the judge model, via a Databricks serving endpoint.

### Notebook compatibility

With rapidly changing libraries such as `langchain`, examples can become outdated rather quickly and will no longer work. For the purposes of demonstration, here are the critical dependencies that are recommended to use to effectively run this notebook:

| Package             | Version     |
|:--------------------|:------------|
| langchain           | **0.1.16**  |
| lanchain-community  | **0.0.33**  |
| langchain-openai    | **0.0.8**   |
| openai              | **1.12.0**  |
| mlflow              | **2.12.1**  |

If you attempt to execute this notebook with different versions, it may function correctly, but it is recommended to use the precise versions above to ensure that your code executes properly.

#### Installing Requirements

Before proceeding with this tutorial, ensure that your versions of the installed packages meet the requirements listed above. 

```bash
    pip install langchain==0.1.16 langchain-community==0.0.33 langchain-openai==0.0.8 openai==1.12.0
```

### Configuration

We need to set our OpenAI API key.

In order to set your private key safely, please be sure to either export your key through a command-line terminal for your current instance, or, for a permanent addition to all user-based sessions, configure your favored environment management configuration file (i.e., .bashrc, .zshrc) to have the following entry:

`OPENAI_API_KEY=<your openai API key>`

In order to run this notebook, using a Databricks hosted Llama2 model, you will need to set your host and personal access token. Please ensure that these are set either using the Databricks SDK ***or*** setting the environment variables:

`DATABRICKS_HOST=<your Databricks workspace URI>`

`DATABRICKS_TOKEN=<your personal access token>`

In [1]:
!pip install langchain langchain-community langchain-openai openai

In [5]:
%load_ext dotenv
%dotenv

import pandas as pd
from langchain.chains import RetrievalQA
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings

import mlflow
# from mlflow.deployments import set_deployments_target
from mlflow.metrics.genai import EvaluationExample, faithfulness, relevance

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


Set the deployment target to "databricks" for use with Databricks served models.

In [2]:
# set_deployments_target("databricks")

## Create a RAG system

Use Langchain and Chroma to create a RAG system that answers questions based on the MLflow documentation.

In [6]:
loader = WebBaseLoader("https://mlflow.org/docs/latest/index.html")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(
    llm=OpenAI(temperature=0),
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    return_source_documents=True,
)

## Evaluate the RAG system using `mlflow.evaluate()`

Create a simple function that runs each input through the RAG chain

In [7]:
def model(input_df):
    answer = []
    for index, row in input_df.iterrows():
        answer.append(qa(row["questions"]))

    return answer

Create an eval dataset

In [8]:
eval_df = pd.DataFrame(
    {
        "questions": [
            "What is MLflow?",
            "How to run mlflow.evaluate()?",
            "How to log_table()?",
            "How to load_table()?",
        ],
    }
)

Create a faithfulness metric using `databricks-llama2-70b-chat` as the judge

In [9]:
# Create a good and bad example for faithfulness in the context of this problem
faithfulness_examples = [
    EvaluationExample(
        input="How do I disable MLflow autologging?",
        output="mlflow.autolog(disable=True) will disable autologging for all functions. In Databricks, autologging is enabled by default. ",
        score=2,
        justification="The output provides a working solution, using the mlflow.autolog() function that is provided in the context.",
        grading_context={
            "context": "mlflow.autolog(log_input_examples: bool = False, log_model_signatures: bool = True, log_models: bool = True, log_datasets: bool = True, disable: bool = False, exclusive: bool = False, disable_for_unsupported_versions: bool = False, silent: bool = False, extra_tags: Optional[Dict[str, str]] = None) → None[source] Enables (or disables) and configures autologging for all supported integrations. The parameters are passed to any autologging integrations that support them. See the tracking docs for a list of supported autologging integrations. Note that framework-specific configurations set at any point will take precedence over any configurations set by this function."
        },
    ),
    EvaluationExample(
        input="How do I disable MLflow autologging?",
        output="mlflow.autolog(disable=True) will disable autologging for all functions.",
        score=5,
        justification="The output provides a solution that is using the mlflow.autolog() function that is provided in the context.",
        grading_context={
            "context": "mlflow.autolog(log_input_examples: bool = False, log_model_signatures: bool = True, log_models: bool = True, log_datasets: bool = True, disable: bool = False, exclusive: bool = False, disable_for_unsupported_versions: bool = False, silent: bool = False, extra_tags: Optional[Dict[str, str]] = None) → None[source] Enables (or disables) and configures autologging for all supported integrations. The parameters are passed to any autologging integrations that support them. See the tracking docs for a list of supported autologging integrations. Note that framework-specific configurations set at any point will take precedence over any configurations set by this function."
        },
    ),
]

faithfulness_metric = faithfulness(
    model="endpoints:/databricks-llama-2-70b-chat", examples=faithfulness_examples
)
print(faithfulness_metric)

EvaluationMetric(name=faithfulness, greater_is_better=True, long_name=faithfulness, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's faithfulness based on the rubric
justification: Your reasoning about the model's faithfulness score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called faithfulness based on the input and output.
A definition of faithfulness and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before completing the task.

Inp

Create a relevance metric using `databricks-llama2-70b-chat` as the judge

In [10]:
# relevance_metric = relevance(model="endpoints:/databricks-llama-2-70b-chat")
# print(relevance_metric)

## Nebius AI Studio - meta-llama/Meta-Llama-3.1-8B-Instruct

In [45]:
from mlflow.deployments import get_deploy_client

llm_client = get_deploy_client("https://localhost:5000")

# llm_client.predict("What is the capital of France")

In [48]:
set_deployments_target("https://localhost:5000")

In [54]:
# llm_client.predict(eval_df)

In [51]:
relevance_metric = relevance(model="endpoints:/ai-studio-chat")
print(relevance_metric)

EvaluationMetric(name=relevance, greater_is_better=True, long_name=relevance, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's relevance based on the rubric
justification: Your reasoning about the model's relevance score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called relevance based on the input and output.
A definition of relevance and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before completing the task.

Input:
{input}

Outpu

In [39]:
# completion = client.chat.completions.create(
#   model="meta-llama/Meta-Llama-3.1-70B-Instruct",
#   messages=[
#     {
#         "role": "user",
#         "content": """Hello!"""
#     }
#   ],
#   temperature=0.0
# )

# relevance_metric = relevance(model=completion)
# print(relevance_metric)

In [52]:
results = mlflow.evaluate(
    model,
    eval_df,
    model_type="question-answering",
    evaluators="default",
    predictions="result",
    extra_metrics=[faithfulness_metric, relevance_metric, mlflow.metrics.latency()],
    evaluator_config={
        "col_mapping": {
            "inputs": "questions",
            "context": "source_documents",
        }
    },
)
print(results.metrics)

2024/10/22 14:26:11 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/10/22 14:26:20 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/10/22 14:26:20 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'exact_match' because it returned None.


  0%|          | 0/1 [00:00<?, ?it/s]

/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3787: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid v

  0%|          | 0/1 [00:00<?, ?it/s]

2024/10/22 14:52:25 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'exact_match' because it returned None.


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

{'latency/mean': 2.2695029973983765, 'latency/variance': 0.5345572663888021, 'latency/p90': 3.083358716964722, 'toxicity/v1/mean': 0.00020220158694428392, 'toxicity/v1/variance': 8.257523553109759e-09, 'toxicity/v1/p90': 0.000301841703185346, 'toxicity/v1/ratio': 0.0, 'flesch_kincaid_grade_level/v1/mean': 6.425000000000001, 'flesch_kincaid_grade_level/v1/variance': 30.366874999999997, 'flesch_kincaid_grade_level/v1/p90': 10.96, 'ari_grade_level/v1/mean': 7.8, 'ari_grade_level/v1/variance': 49.864999999999995, 'ari_grade_level/v1/p90': 14.65, 'faithfulness/v1/mean': nan, 'faithfulness/v1/variance': nan, 'relevance/v1/mean': nan, 'relevance/v1/variance': nan}


In [53]:
results.tables["eval_results_table"]

,questions,outputs,source_documents,latency,token_count,toxicity/v1/score,flesch_kincaid_grade_level/v1/score,ari_grade_level/v1/score,faithfulness/v1/score,faithfulness/v1/justification,relevance/v1/score,relevance/v1/justification
0,What is MLflow?,MLflow is an open-source platform designed to...,"[{'id': None, 'lc_attributes': {}, 'lc_secrets...",1.792353,48,0.000140,12.1,17.2,NaN,Failed to score model on payload. Error: API r...,NaN,Failed to score model on payload. Error: API r...
1,How to run mlflow.evaluate()?,"To run mlflow.evaluate(), you will need to ha...","[{'id': None, 'lc_attributes': {}, 'lc_secrets...",3.293630,74,0.000358,8.0,8.0,NaN,Failed to score model on payload. Error: API r...,NaN,Failed to score model on payload. Error: API r...
2,How to log_table()?,"I don't know, as there is no mention of a fun...","[{'id': None, 'lc_attributes': {}, 'lc_secrets...",2.592726,44,0.000140,8.3,8.7,NaN,Failed to score model on payload. Error: API r...,NaN,Failed to score model on payload. Error: API r...
3,How to load_table()?,I don't know.,"[{'id': None, 'lc_attributes': {}, 'lc_secrets...",1.399303,5,0.000171,-2.7,-2.7,NaN,Failed to score model on payload. Error: API r...,NaN,Failed to score model on payload. Error: API r...
